In [106]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [3]:
loader=PyPDFDirectoryLoader("pdf")

In [107]:
data=loader.load()

In [6]:
data[0].page_content

'YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known\nreal-time object detectors with 30 FPS or higher on GPU\nV100. YOLOv7-E6 object detector (56 FPS V100, 55.9%\nAP) outperforms both transformer-based detector SWIN-\nL Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by\n509% in speed and 2% in accuracy, and convolutional-\nbased detector ConvNeXt-XL Cascade-Mask R-CNN (8.6\nFPS A100, 55.2% AP) by 551% in speed and 0.7% AP\nin accuracy, as well as YOLOv7 outperforms: YOLOR,\nYOLOX, Scaled-YOLOv4, YOLOv5, DETR, Deformable\nDETR, DINO-5scale-R50, ViT-Adapter-B and many other\nobject det

In [108]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [109]:
text_chunks = text_splitter.split_documents(data)

In [35]:
text_chunks[0].metadata['page']

0

In [10]:
text_chunks[0].page_content

'YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known'

In [11]:
len(text_chunks)

152

In [110]:
i=0
j=1
text_chunks_truncated=[]
for d in text_chunks:
    if i < 96:
        new_entry={
            'id':j,
            'content':d.page_content
        }
        text_chunks_truncated.append(new_entry)
        i=i+1
        j=j+1
print(i) 

96


In [77]:
text_chunks_truncated[0]

{'id': 1,
 'content': 'YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known'}

In [111]:
len(text_chunks_truncated)

96

In [72]:
text_chunks_truncated[10]

Document(metadata={'source': 'pdf\\yolov7.pdf', 'page': 1}, page_content='strategies applicable to layers in different networks with the\nconcept of gradient propagation path, and propose planned\nre-parameterized model. In addition, when we discover that\nwith dynamic label assignment technology, the training of\nmodel with multiple output layers will generate new issues.\nThat is: “How to assign dynamic targets for the outputs of\ndifferent branches?” For this problem, we propose a new\nlabel assignment method called coarse-to-ﬁne lead guided\nlabel assignment.')

In [79]:
text_chunks_truncated[0]['id']

1

In [80]:
text_chunks_truncated[0]['content']

'YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known'

In [47]:
# Define a sample dataset where each item has a unique ID and piece of text
data1 = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
    {"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}
]

In [112]:
from dotenv import load_dotenv

In [113]:
# Load environment variables from the .env file
load_dotenv()

# Access the environment variables just like you would with os.environ
key = os.getenv("OPENAI_API_KEY")
PC_KEY=os.getenv("PINECONE_API_KEY")

In [114]:
embedding=OpenAIEmbeddings()

In [15]:
len(embedding.embed_query("How are you"))

1536

In [16]:
len(embedding.embed_query("hi i am fine"))

1536

In [115]:
# Import the Pinecone library
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=PC_KEY)

In [116]:
# Convert the text into numerical vectors that Pinecone can index
# Convert the text into numerical vectors that Pinecone can index
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['content'] for d in text_chunks_truncated],
    parameters={"input_type": "passage", "truncate": "END"}
)

print(embeddings)

EmbeddingsList(
  model='multilingual-e5-large',
  data=[
    {'values': [0.024993896484375, 0.00855255126953125, ..., -0.0210418701171875, 0.00775146484375]},
    {'values': [0.02288818359375, 0.01258087158203125, ..., -0.0159912109375, 0.0030307769775390625]},
    ... (92 more embeddings) ...,
    {'values': [0.006866455078125, -0.0177001953125, ..., -0.033447265625, 0.0217437744140625]},
    {'values': [0.01068115234375, 0.0234527587890625, ..., -0.01419830322265625, 0.01175689697265625]}
  ],
  usage={'total_tokens': 12734}
)


In [117]:
# Create a serverless index
index_name = "example-index"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [119]:
# Target the index where you'll store the vector embeddings
index = pc.Index("example-index")

# Prepare the records for upsert
# Each contains an 'id', the embedding 'values', and the original text as 'metadata'
records = []
for d, e in zip(text_chunks_truncated, embeddings):
    records.append({
        "id": str(d['id']),
        "values": e['values'],
        "metadata": {'text': d['content']}
    })

# Upsert the records into the index
index.upsert(
    vectors=records,
    namespace="example-namespace"
)

upserted_count: 96

In [120]:
print(index.describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'example-namespace': {'vector_count': 96}},
 'total_vector_count': 96}


In [89]:
 #Define your query
query = "what is yolo?"

# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

# Search the index for the three most similar vectors
results = index.query(
    namespace="example-namespace",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': '61',
              'metadata': {'text': 'E6E. Since YOLOv7-tiny is an edge '
                                   'GPU-oriented archi-\n'
                                   'tecture, it will use leaky ReLU as '
                                   'activation function. As\n'
                                   'for other models we use SiLU as activation '
                                   'function. We\n'
                                   'will describe the scaling factor of each '
                                   'model in detail in\n'
                                   'Appendix.\n'
                                   '6'},
              'score': 0.82340646,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '45',
              'metadata': {'text': 'For example, YOLO [61] use IoU of '
                                   'prediction of bounding\n'
                                   'box regression and ground

In [99]:
results['matches'][0]['metadata']['text']

'E6E. Since YOLOv7-tiny is an edge GPU-oriented archi-\ntecture, it will use leaky ReLU as activation function. As\nfor other models we use SiLU as activation function. We\nwill describe the scaling factor of each model in detail in\nAppendix.\n6'

In [124]:
pc.delete_index(index_name)

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'ca444555f52fe5f6e0b2d2fa48096bd4', 'Date': 'Thu, 07 Nov 2024 22:29:51 GMT', 'Server': 'Google Frontend', 'Content-Length': '88', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource example-index not found"},"status":404}
